In [1]:
from lxml import etree

In [2]:
college = 'Algonquin'

infile = 'input/' + college +'_invoices_20211123.xml'
outfile = 'output/' + college +'_invoices_fixed_20211123.xml'

In [3]:
# Specify the XMLParser with recover option to catch any '&' that 
# have not been properly escaped in the source file.
tree = etree.parse(infile, parser=etree.XMLParser(recover=True))
amounts = tree.xpath('//exchangeRate/../vendorAmount')
print(len(amounts))

61


In [5]:
for amount in amounts:
    amount.attrib['currency']='USD'
tree.write(outfile, pretty_print=True)